### Extract e mais algumas mudanças de transformação

In [2]:
import pandas as pd

In [55]:
algora = pd.read_csv('algoraquant.csv')
algora['vl_valor'] = algora['vl_valor'].fillna(0)
algora['vl_vpa'] = algora['vl_vpa'].fillna(0)

In [56]:
import pandas as pd

# Exemplo de DataFrame
# algora = pd.DataFrame({
#     'vl_vpa': ['1,8100', '2,0000', '1.056.2292', 3.5000],  # Valores mistos
#     'vl_valor': ['4,33%', '2,50%', '5,00%']
# })

# Função para calcular o novo valor
def calcular_valor_normal(row):
    # Limpa e formata 'vl_vpa' antes de converter para float
    vpa_str = str(row['vl_vpa']).replace('.', '').replace(',', '.').strip()  # Remove milhar e converte decimal
    try:
        vpa = float(vpa_str)  # Converte para float
    except ValueError:
        return None  # Retorna None se a conversão falhar
    
    # Remove o '%' e substitui a vírgula por ponto, depois converte para float
    variacao = float(row['vl_valor'].replace('%', '').replace(',', '.'))
    
    # Calcula o novo valor
    novo_valor = vpa * (1 + variacao / 100)
    return novo_valor

# Filtra as linhas onde 'vl_valor' contém '%'
filtro_percentual = algora['vl_valor'].str.contains('%', na=False)

# Aplica o cálculo apenas para essas linhas
algora.loc[filtro_percentual, 'vl_valor'] = algora.loc[filtro_percentual].apply(calcular_valor_normal, axis=1)

In [62]:
import pandas as pd

# Supondo que você já tem o DataFrame 'algora'

# Lista de colunas para as quais queremos aplicar a conversão
colunas_para_convertir = [
    'vl_avaliacao_media_ativos_fundo', 'vl_quantidade_imoveis', 'vl_valor',
    'vl_liquidez_media_diaria', 'vl_pvp', 'vl_dividendo', 'vl_yeld',
    'vl_soma_yield_3m', 'vl_soma_yield_6m', 'vl_soma_yield_12m',
    'vl_media_yield_3m', 'vl_media_yield_6m', 'vl_media_yield_12m',
    'vl_soma_yield_ano_corrente', 'vl_variacao_cotacao_mes', 'vl_rentabilidade_mes',
    'vl_rentabilidade', 'vl_patrimonio', 'vl_vpa', 'vl_pvpa', 'vl_vpa_yield',
    'vl_vpa_change', 'vl_vpa_rent_m', 'vl_vpa_rent', 'vl_ativos',
    'vl_volatility', 'vl_numero_cotista', 'vl_vacancia_fisica', 'vl_vacancia_financeira'
]

# Função para converter a coluna para float
def converter_para_float(coluna):
    try:
        # Remove os pontos (separadores de milhar)
        coluna = coluna.str.replace('.', '', regex=False)
        # Substitui a vírgula por ponto (separador decimal)
        coluna = coluna.str.replace(',', '.', regex=False)
        # Converte para float
        return coluna.astype(float)
    except Exception as e:
        print(f"Erro ao converter coluna: {e}")
        return coluna  # Retorna a coluna original em caso de erro

# Aplica a conversão para cada coluna na lista
for coluna in colunas_para_convertir:
    if coluna in algora.columns:
        algora[coluna] = converter_para_float(algora[coluna])


Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao converter coluna: Can only use .str accessor with string values!
Erro ao converter coluna: could not convert string to float: '1.19 %'
Erro ao converter coluna: could not convert string to float: '3.46 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '1.15 %'
Erro ao converter coluna: could not convert string to float: '1.37 %'
Erro ao converter coluna: could not convert string to float: '1.37 %'
Erro ao converter coluna: could not convert string to float: '6.85 %'
Erro ao converter coluna: could not convert string to float: '1.27 %'
Erro ao converter coluna: could not convert string to float: '2.48 %'
Erro ao converter coluna: could not convert string to float: '12.91 %'
Erro ao co

Pronto estamos prontos para começar as nossas análises exploratórias, aqui deixamos apenas valores decimais facieís de serem tratados, 
ou seja ... que não nenhum tipo de porcentagem envolvido, ficando cerca de 10 colunas de valores a serem análisados de acordo com a estatística

In [64]:
algora = algora.drop(columns=['Unnamed: 0'])

# Análise de Dados de Fundos Imobiliários

Nesta análise, exploraremos um DataFrame que contém informações detalhadas sobre fundos imobiliários. Cada coluna do DataFrame fornece insights importantes sobre o fundo e sua performance. Abaixo está uma explicação detalhada sobre o que cada variável representa e sua importância na análise de fundos imobiliários.

## Variáveis do DataFrame

### `nm_titulo`
- **Descrição**: Nome do título ou do fundo imobiliário.
- **Importância**: Identifica o fundo específico que está sendo analisado. Essencial para distinguir entre diferentes fundos e comparar suas características.

### `vl_media_taxa_homicidios`
- **Descrição**: Taxa média de homicídios na região onde o fundo possui ativos.
- **Importância**: Pode afetar a valorização dos imóveis e a atratividade do investimento na área. Importante para avaliar riscos associados à localização dos imóveis.

### `vl_avaliacao_media_ativos_fundo`
- **Descrição**: Avaliação média dos ativos do fundo.
- **Importância**: Indica o valor médio dos ativos que o fundo possui. Fundamental para entender a valorização e o valor de mercado do fundo.

### `vl_quantidade_imoveis`
- **Descrição**: Quantidade de imóveis que compõem o fundo.
- **Importância**: Reflete a diversificação do fundo. Fundos com mais imóveis podem ter menor risco, pois a performance não depende de um único ativo.

### `nm_setor`
- **Descrição**: Setor ao qual o fundo imobiliário pertence (ex.: comercial, residencial, industrial).
- **Importância**: Ajuda a identificar o foco do fundo e avaliar como diferentes setores estão se comportando no mercado imobiliário.

### `vl_valor`
- **Descrição**: Valor do fundo ou do ativo.
- **Importância**: Medida crucial para avaliar o tamanho e a importância do fundo no mercado. Pode ser usado para comparar com outros fundos.

### `vl_liquidez_media_diaria`
- **Descrição**: Liquidez média diária do fundo.
- **Importância**: Indica a facilidade com que o fundo pode ser comprado ou vendido no mercado. Liquidez alta é geralmente preferível, pois permite maior flexibilidade para os investidores.

### `vl_pvp`
- **Descrição**: Preço sobre o valor patrimonial (P/VPA).
- **Importância**: Ratio que compara o preço de mercado do fundo com seu valor patrimonial. Um P/VPA abaixo de 1 pode indicar que o fundo está sendo negociado a um desconto.

### `vl_dividendo`
- **Descrição**: Valor do dividendo pago pelo fundo.
- **Importância**: Fonte de rendimento para os investidores. Importante para avaliar a atratividade do fundo em termos de retorno sobre o investimento.

### `vl_yeld`
- **Descrição**: Rentabilidade ou yield do fundo.
- **Importância**: Medida de retorno anualizada do fundo. Ajuda os investidores a entenderem o rendimento do investimento em relação ao valor investido.

### `vl_soma_yield_3m`, `vl_soma_yield_6m`, `vl_soma_yield_12m`
- **Descrição**: Soma dos yields dos últimos 3, 6 e 12 meses, respectivamente.
- **Importância**: Avalia a performance do fundo em diferentes horizontes de tempo. Permite a análise de tendências e a comparação com benchmarks.

### `vl_media_yield_3m`, `vl_media_yield_6m`, `vl_media_yield_12m`
- **Descrição**: Média do yield dos últimos 3, 6 e 12 meses, respectivamente.
- **Importância**: Fornece uma visão geral da performance média do fundo ao longo do tempo. Útil para detectar variações e tendências.

### `vl_soma_yield_ano_corrente`
- **Descrição**: Soma do yield no ano corrente.
- **Importância**: Avalia o desempenho do fundo em termos de rendimento até a data atual no ano corrente.

### `vl_variacao_cotacao_mes`
- **Descrição**: Variação da cotação do fundo no mês.
- **Importância**: Medida da volatilidade e das flutuações de preço do fundo ao longo do mês.

### `vl_rentabilidade_mes`
- **Descrição**: Rentabilidade mensal do fundo.
- **Importância**: Avalia o retorno do fundo em uma base mensal, permitindo uma análise detalhada de sua performance recente.

### `vl_rentabilidade`
- **Descrição**: Rentabilidade total do fundo.
- **Importância**: Medida abrangente do retorno do fundo ao longo de um período específico.

### `vl_patrimonio`
- **Descrição**: Patrimônio total do fundo.
- **Importância**: Reflete o valor total dos ativos líquidos do fundo. Importante para avaliar a escala e a capacidade do fundo de gerar retornos.

### `vl_vpa`
- **Descrição**: Valor patrimonial por ação.
- **Importância**: Representa o valor contábil do fundo dividido pelo número de ações em circulação. Ajuda a avaliar a justiça do preço do fundo no mercado.

### `vl_pvpa`
- **Descrição**: Preço sobre o valor patrimonial por ação.
- **Importância**: Ratio semelhante ao P/VPA, mas focado em cada ação individual. Útil para investidores que compram ações individuais do fundo.

### `vl_vpa_yield`
- **Descrição**: Yield baseado no valor patrimonial por ação.
- **Importância**: Avalia o rendimento do fundo em relação ao seu valor patrimonial por ação.

### `vl_vpa_change`
- **Descrição**: Mudança no valor patrimonial por ação.
- **Importância**: Mostra a variação do valor patrimonial ao longo do tempo. Importante para entender a tendência de valorização ou desvalorização do fundo.

### `vl_vpa_rent_m`, `vl_vpa_rent`
- **Descrição**: Rentabilidade baseada no valor patrimonial por ação.
- **Importância**: Avalia o retorno do fundo com base no seu valor patrimonial.

### `vl_ativos`
- **Descrição**: Valor total dos ativos do fundo.
- **Importância**: Avalia a capacidade do fundo de gerar retorno a partir dos seus ativos.

### `vl_volatility`
- **Descrição**: Volatilidade do fundo.
- **Importância**: Medida do risco associado ao fundo. Alta volatilidade indica maior risco, enquanto baixa volatilidade sugere menor risco.

### `vl_numero_cotista`
- **Descrição**: Número de cotistas no fundo.
- **Importância**: Indica a popularidade e a base de investidores do fundo.

### `dt_inclusao`
- **Descrição**: Data de inclusão dos dados no DataFrame.
- **Importância**: Permite rastrear quando os dados foram adicionados e ajuda na análise temporal.

### `vl_vacancia_fisica`
- **Descrição**: Taxa de vacância física dos imóveis do fundo.
- **Importância**: Reflete a porcentagem de imóveis não alugados. Taxas altas podem indicar problemas na gestão de propriedades.

### `vl_vacancia_financeira`
- **Descrição**: Taxa de vacância financeira dos imóveis do fundo.
- **Importância**: Mede a porcentagem de receita potencial que não está sendo gerada devido a imóveis vagos. Importante para avaliar a eficiência na geração de renda.


In [65]:
algora.describe()

,vl_media_taxa_homicidios,vl_avaliacao_media_ativos_fundo,vl_quantidade_imoveis,vl_valor,vl_liquidez_media_diaria,vl_pvp,vl_dividendo,vl_vpa,vl_pvpa,vl_volatility,vl_numero_cotista,valor_decimal
count,490.000000,490.000000,490.000000,9037.000000,5.883000e+03,2294.000000,9899.000000,9.445000e+03,8804.000000,445.000000,4.900000e+02,9900.0
mean,11.487502,0.850446,3.193878,440.740539,4.400724e+05,8.033819,6.024203,1.031683e+08,inf,56.608360,3.022137e+04,0.0
std,23.832172,1.639020,9.006994,4228.455721,1.388703e+06,106.980097,86.938202,4.304699e+08,NaN,127.894745,8.447712e+04,0.0
min,0.000000,0.000000,0.000000,0.210000,0.000000e+00,0.010000,0.000000,-1.662371e+07,-3.820000,0.370000,0.000000e+00,0.0
25%,0.000000,0.000000,0.000000,70.720000,2.305000e+03,0.752500,0.394150,9.382000e+01,0.860000,11.720000,1.150000e+02,0.0
50%,0.000000,0.000000,0.000000,96.000000,2.838000e+04,0.930000,0.700000,1.067209e+02,0.993850,16.510000,2.618000e+03,0.0
75%,0.000000,0.000000,2.000000,119.250000,2.285800e+05,1.000000,1.100000,1.464580e+03,1.191225,38.350000,1.595925e+04,0.0
max,119.900000,5.000000,72.000000,94794.810000,3.180000e+07,1961.670000,5302.020000,7.984130e+09,inf,961.290000,1.142848e+06,0.0


Primeira missão: remover outliers